# About
This notebook will produce all the electricity data for step 1.1B of the data processing for Module 1.

**Required user input**

Update the cell below once each for the full years of 2019, 2020, 2021, and run the entire notebook for each.


In [ ]:
year = '2021' # Update year - this is the year of data you are working on.

In [ ]:
# Don't change these.
output_directory='Step_1_1_Outputs'

first_date = year+'-01-01'
last_date = year+'-12-31'
daily_filename = 'Step_1_1B_Elec_'+year+'_daily_from_daily' # Filename shouldn't include '.csv'
hh_subdir='Step_1_1B_Elec_'+year+'_hh' # Filename shouldn't include '.csv'

# Code

In [ ]:
import os
import pandas as pd
from data_picker_edition_4_v02 import SerlDataSelector
import locations
import datetime
now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Run with Observatory data version: {locations.serl_data_version}\nRun on {now}")

## Daily data 

In [ ]:
#Initialise the data picker
selector = SerlDataSelector()

In [ ]:
# Create the data
selector.load_data(res='daily',
                   usecols=['PUPRN','Read_date_effective_local','Valid_read_time','Elec_act_imp_flag','Elec_act_imp_d_Wh'],
                   filter_Elec_act_imp_flag=[1], # Include only valid daily reads
                   filter_Valid_read_time=True, # For edition 4 data, this is not needed (but does no harm)
                   first_date = first_date,
                   last_date = last_date,
                   inc_time_change_days=True) 
# Save it
selector.save_data(output_filename=daily_filename,
                   output_directory=output_directory,
                   save_method='single_file')

## Half-hourly data

### Load the data with the data picker

In [ ]:
# Initialise the data picker
selector = SerlDataSelector('hh')

In [ ]:
selector.load_data(res='hh',
                   usecols=['PUPRN','Read_date_time_UTC','Read_date_effective_local','Read_date_time_local','Valid_read_time',
                            'Elec_act_imp_flag','Elec_act_exp_flag','Elec_act_imp_hh_Wh','Elec_act_exp_hh_Wh'],
                   filter_Elec_act_imp_flag=[1], # Include only valid reads
                   filter_Elec_act_exp_flag=[1,2], # Include valid reads AND readings for homes with no export meter
                   filter_Valid_read_time=True,
                   first_date = first_date,
                   last_date = last_date,
                   inc_time_change_days=True,
                   add_local_time_cols = False,
                   add_net_electricity_column = True) # Calculates it for all selected rows, taking export values to be zero if they are NaN (this is a valid assumption, as they are flagged as either valid reads or from homes with no export meter.
# Save it
selector.save_data(output_filename=hh_subdir,
                   output_directory=output_directory,
                   save_method='per_home')